In [1]:
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# script to resize the images present in the dataset
# vgg16 model requires input images to be of size 224x224

train_dir = "Variant-a(Multiclass Classification)/train"
val_dir = "Variant-a(Multiclass Classification)/val"
test_dir = "Variant-a(Multiclass Classification)/test"
img_size = (224, 224)  
batch_size = 32       

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Multiclass classification
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Keep order for evaluation
)

# Check class indices (important for mapping predictions)
print("Class Indices: ", train_generator.class_indices)


Found 3162 images belonging to 8 classes.
Found 902 images belonging to 8 classes.
Found 461 images belonging to 8 classes.
Class Indices:  {'Early_blight': 0, 'Healthy': 1, 'Late_blight': 2, 'Leaf Miner': 3, 'Magnesium Deficiency': 4, 'Nitrogen Deficiency': 5, 'Pottassium Deficiency': 6, 'Spotted Wilt Virus': 7}


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout


In [5]:
# base model being called
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
model = Sequential([
    base_model,                  
    Flatten(),                      
    Dense(256, activation="relu"),  
    Dropout(0.5),                   
    Dense(128, activation="relu"),  
    Dense(train_generator.num_classes, activation="softmax") 
])

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,171,400 (80.76 MB)

 Trainable params: 6,456,712 (24.63 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
epochs = 20
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size

In [11]:
#saving the best model based on validation accuracy

#checkpoint = ModelCheckpoint(
#   "vgg16_plant_disease_model.keras",  #save model in this file
#   monitor="val_accuracy",          
#   save_best_only=True,             # Save only the best-performing model
#   mode="max",                     
#   verbose=1 )

#stop training if validation loss does not improve for continuous 5 epochs. restore the best weights
early_stop = EarlyStopping(
    monitor="val_loss", 
    patience=5, 
    restore_best_weights=True
)

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[early_stop],
    verbose=1 
)

Epoch 1/10


c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2883 - loss: 1.8783
Epoch 1: val_accuracy improved from -inf to 0.54464, saving model to vgg16_plant_disease_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 312s 3s/step - accuracy: 0.2892 - loss: 1.8767 - val_accuracy: 0.5446 - val_loss: 1.3794
Epoch 2/10
 1/98 ━━━━━━━━━━━━━━━━━━━━ 4:15 3s/step - accuracy: 0.5625 - loss: 1.5216

c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_accuracy did not improve from 0.54464
98/98 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5625 - loss: 1.5216 - val_accuracy: 0.5000 - val_loss: 1.5821
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5009 - loss: 1.4272
Epoch 3: val_accuracy improved from 0.54464 to 0.57924, saving model to vgg16_plant_disease_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 301s 3s/step - accuracy: 0.5011 - loss: 1.4269 - val_accuracy: 0.5792 - val_loss: 1.2117
Epoch 4/10
 1/98 ━━━━━━━━━━━━━━━━━━━━ 4:12 3s/step - accuracy: 0.5000 - loss: 1.3653
Epoch 4: val_accuracy improved from 0.57924 to 0.83333, saving model to vgg16_plant_disease_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5000 - loss: 1.3653 - val_accuracy: 0.8333 - val_loss: 0.8191
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5702 - loss: 1.2468
Epoch 5: val_accuracy did not improve from 0.83333
98/98 ━━━━━━━━━━━━━━━━━━━━ 298s 3s/step - accuracy: 0.5702 - loss: 1.2468 - val_accuracy: 0